In [20]:
import sys
import os

sys.path.append(os.path.abspath(".."))

In [21]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from models.lstm_cnn_attention import LSTMCNNAttention

# For reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [22]:
# Sanity check for LSTM-CNN-Attention model

model = LSTMCNNAttention()

x = torch.randn(8, 75, 3)  # batch of 8 samples
y = model(x)

print("Output shape:", y.shape)

Output shape: torch.Size([8, 3])


In [23]:
# Sanity check for Autoencoder 

from models.sequence_autoencoder import SequenceAutoencoder

ae = SequenceAutoencoder()

x = torch.randn(4, 75, 3)
recon = ae(x)

print("Input shape:", x.shape)
print("Reconstructed shape:", recon.shape)

Input shape: torch.Size([4, 75, 3])
Reconstructed shape: torch.Size([4, 75, 3])


In [24]:
# Load pre-generated dataset
X_train = np.load("../data/X_train.npy")
y_train = np.load("../data/y_train.npy")

X_val = np.load("../data/X_val.npy")
y_val = np.load("../data/y_val.npy")

print("Train shape:", X_train.shape, y_train.shape)
print("Val shape:", X_val.shape, y_val.shape)

Train shape: (10500, 75, 3) (10500,)
Val shape: (2250, 75, 3) (2250,)


In [25]:
# Convert to PyTorch tensors
'''
    Why?
    PyTorch models only work with tensors
    Labels must be long for classification
'''

X_train_t = torch.tensor(X_train, dtype = torch.float32)
y_train_t = torch.tensor(y_train, dtype = torch.long)

X_val_t = torch.tensor(X_val, dtype = torch.float32)
y_val_t = torch.tensor(y_val, dtype = torch.long)

In [26]:
# Model
model = LSTMCNNAttention()

# Loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

# Training params
EPOCHS = 15
BATCH_SIZE = 64

Simple. No tuning yet

In [27]:
def train_one_epoch(model, X, y, optimizer, criterion, batch_size):

    model.train()
    total_loss = 0
    correct = 0
    num_batches = (len(X) + batch_size - 1) // batch_size

    for i in range(0, len(X), batch_size):
        xb = X[i:i + batch_size]
        yb = y[i:i + batch_size]

        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs, yb)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(dim = 1)
        correct += (preds == yb).sum().item()

        # Print progress
        print(f"Batch {i//batch_size + 1}/{num_batches} - Loss: {loss.item():.4f}")

    acc = correct / len(X)
    return total_loss, acc

In [28]:
# Validation Loop

def evaluate(model, X, y, criterion):
    model.eval()
    total_loss = 0
    correct = 0

    with torch.no_grad():
        outputs = model(X)
        loss = criterion(outputs, y)

        total_loss = loss.item()
        preds = outputs.argmax(dim = 1)
        correct = (preds == y).sum().item()

    acc = correct / len(X)
    return total_loss, acc

In [ ]:
# Train Model

for epoch in range(EPOCHS):
    
    train_loss, train_acc = train_one_epoch(
        model, X_train_t, y_train_t, optimizer, criterion, BATCH_SIZE
    )

    val_loss, val_acc = evaluate(
        model, X_val_t, y_val_t, criterion
    )

    print(
        f"Epoch {epoch+1}/{EPOCHS} | "
        f"Train Acc: {train_acc:.3f} | "
        f"Val Acc: {val_acc:.3f}"
    )

```
Batch 1/165 - Loss: 1.0990
Batch 2/165 - Loss: 1.0990
Batch 3/165 - Loss: 1.0894
Batch 4/165 - Loss: 1.0835
Batch 5/165 - Loss: 1.0903
...
Batch 163/165 - Loss: 0.0109
Batch 164/165 - Loss: 0.0107
Batch 165/165 - Loss: 0.0208
```

Epoch 15/15 | Train Acc: 0.986 | Val Acc: 0.997

On synthetic data, you should see:

- accuracy quickly rise above 90%
- validation track training closely

That confirms:
- dataset is usable
- model is learning
- pipeline is correct

In [30]:
# Load test data
X_test = np.load("../data/X_test.npy")
y_test = np.load("../data/y_test.npy")

X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.long)

print("Test shape:", X_test_t.shape, y_test_t.shape)

Test shape: torch.Size([2250, 75, 3]) torch.Size([2250])


In [31]:
# Evaluate on test set
test_loss, test_acc = evaluate(
    model, X_test_t, y_test_t, criterion
)

print(f"Test Accuracy: {test_acc:.3f}")

Test Accuracy: 0.998


You should expect:
- Test accuracy ≈ validation accuracy
- Slight drop is okay

In [32]:
# Confusion Matrix

from sklearn.metrics import confusion_matrix, classification_report

model.eval()
with torch.no_grad():
    outputs = model(X_test_t)
    preds = outputs.argmax(dim=1).cpu().numpy()

cm = confusion_matrix(y_test, preds)
print("Confusion Matrix:\n", cm)

print("\nClassification Report:")
print(classification_report(y_test, preds, target_names=[
    "Light Braking", "Normal Braking", "Emergency Braking"
]))

Confusion Matrix:
 [[775   4   0]
 [  0 711   0]
 [  0   0 760]]

Classification Report:
                   precision    recall  f1-score   support

    Light Braking       1.00      0.99      1.00       779
   Normal Braking       0.99      1.00      1.00       711
Emergency Braking       1.00      1.00      1.00       760

         accuracy                           1.00      2250
        macro avg       1.00      1.00      1.00      2250
     weighted avg       1.00      1.00      1.00      2250



In [33]:
# Save trained model
torch.save(model.state_dict(), "../models/lstm_cnn_attention_baseline.pth")
print("Model saved successfully.")

Model saved successfully.


Files after saving this model 

models/
- lstm_cnn_attention.py
- lstm_cnn_attention_baseline.pth


## Autoencoder

In [34]:
ae = SequenceAutoencoder()

ae_criterion = nn.MSELoss()
ae_optimizer = optim.Adam(ae.parameters(), lr=1e-3)

AE_EPOCHS = 20
AE_BATCH_SIZE = 64

In [35]:
# Train Autoencoder 
def train_autoencoder(model, X, optimizer, criterion, batch_size):
    model.train()
    total_loss = 0

    for i in range(0, len(X), batch_size):
        xb = X[i:i+batch_size]

        optimizer.zero_grad()
        recon = model(xb)
        loss = criterion(recon, xb)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(X)

In [36]:
for epoch in range(AE_EPOCHS):
    loss = train_autoencoder(
        ae, X_train_t, ae_optimizer, ae_criterion, AE_BATCH_SIZE
    )

    print(f"AE Epoch {epoch+1}/{AE_EPOCHS} | Reconstruction Loss: {loss:.6f}")

AE Epoch 1/20 | Reconstruction Loss: 1.573429
AE Epoch 2/20 | Reconstruction Loss: 0.075039
AE Epoch 3/20 | Reconstruction Loss: 0.025374
AE Epoch 4/20 | Reconstruction Loss: 0.017062
AE Epoch 5/20 | Reconstruction Loss: 0.004242
AE Epoch 6/20 | Reconstruction Loss: 0.000468
AE Epoch 7/20 | Reconstruction Loss: 0.000388
AE Epoch 8/20 | Reconstruction Loss: 0.000373
AE Epoch 9/20 | Reconstruction Loss: 0.000358
AE Epoch 10/20 | Reconstruction Loss: 0.000344
AE Epoch 11/20 | Reconstruction Loss: 0.000329
AE Epoch 12/20 | Reconstruction Loss: 0.000314
AE Epoch 13/20 | Reconstruction Loss: 0.000300
AE Epoch 14/20 | Reconstruction Loss: 0.000285
AE Epoch 15/20 | Reconstruction Loss: 0.000271
AE Epoch 16/20 | Reconstruction Loss: 0.000258
AE Epoch 17/20 | Reconstruction Loss: 0.000245
AE Epoch 18/20 | Reconstruction Loss: 0.000233
AE Epoch 19/20 | Reconstruction Loss: 0.000221
AE Epoch 20/20 | Reconstruction Loss: 0.000210


In [37]:
# Save trained Autoencoder 
torch.save(ae.state_dict(), "../models/sequence_autoencoder.pth")

print("Autoencoder saved.")

Autoencoder saved.


In [38]:
# AE + Classifier Sanity check
from models.lstm_cnn_attention import AE_LSTMCNNAttention

ae_model = AE_LSTMCNNAttention()

x = torch.randn(4, 75, 3)
y = ae_model(x)

print("Output shape:", y.shape)

Output shape: torch.Size([4, 3])


In [39]:
# Train the integrated model
ae_classifier = AE_LSTMCNNAttention()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, ae_classifier.parameters()),
    lr = 1e-3
)

EPOCHS = 15
BATCH_SIZE = 64

In [ ]:
for epoch in range(EPOCHS):
    
    train_loss, train_acc = train_one_epoch(
        ae_classifier, X_train_t, y_train_t,
        optimizer, criterion, BATCH_SIZE
    )

    val_loss, val_acc = evaluate(
        ae_classifier, X_val_t, y_val_t, criterion
    )

    print(
        f"[AE+CLS] Epoch {epoch+1}/{EPOCHS} | "
        f"Train Acc: {train_acc:.3f} | "
        f"Val Acc: {val_acc:.3f}"
    )

```
Batch 1/165 - Loss: 1.0949
Batch 2/165 - Loss: 1.0915
Batch 3/165 - Loss: 1.0816
Batch 4/165 - Loss: 1.0733
Batch 5/165 - Loss: 1.0732
...
Batch 163/165 - Loss: 0.0027
Batch 164/165 - Loss: 0.0032
Batch 165/165 - Loss: 0.0004
```

[AE+CLS] Epoch 15/15 | Train Acc: 0.996 | Val Acc: 0.994

## Test-set evaluation for AE + Classifier

In [ ]:
# Load test data
X_test = np.load("../data/X_test.npy")
y_test = np.load("../data/y_test.npy")

X_test_t = torch.tensor(X_test, dtype = torch.float32)
y_test_t = torch.tensor(y_test, dtype = torch.long)

print("Test set shape:", X_test_t.shape, y_test_t.shape)

Test set shape: torch.Size([2250, 75, 3]) torch.Size([2250])


In [42]:
test_loss, test_acc = evaluate(
    ae_classifier, X_test_t, y_test_t, criterion
)

print(f"[AE+CLS] Test Accuracy: {test_acc:.4f}")

[AE+CLS] Test Accuracy: 0.9929


In [ ]:
# Confusion Matrix

from sklearn.metrics import confusion_matrix, classification_report

ae_classifier.eval()
with torch.no_grad():
    outputs = ae_classifier(X_test_t)
    preds = outputs.argmax(dim = 1).cpu().numpy()

cm = confusion_matrix(y_test, preds)
print("Confusion Matrix (AE+CLS):\n", cm)

print("\nClassification Report (AE+CLS):")
print(classification_report(
    y_test,
    preds,
    target_names = ["Light Braking", "Normal Braking", "Emergency Braking"]
))

Confusion Matrix (AE+CLS):
 [[779   0   0]
 [ 16 695   0]
 [  0   0 760]]

Classification Report (AE+CLS):
                   precision    recall  f1-score   support

    Light Braking       0.98      1.00      0.99       779
   Normal Braking       1.00      0.98      0.99       711
Emergency Braking       1.00      1.00      1.00       760

         accuracy                           0.99      2250
        macro avg       0.99      0.99      0.99      2250
     weighted avg       0.99      0.99      0.99      2250



---
---

Test results:

Test Accuracy: 99.29%

Confusion Matrix:

    Light     → almost perfect
    Normal    → small confusion with Light (16 samples)
    Emergency → perfect

This tells us three important things:

✅ (1) No train–test leakage

If there was leakage:
- test accuracy would be ~100%
- confusion matrix would be perfectly diagonal

But we do have:
- small, realistic confusion (Normal ↔ Light)
- slightly lower test accuracy than val

This is healthy.

✅ (2) Emergency braking is learned robustly

This is critical for both real-world relevance & research credibility

Emergency braking:
- Precision = 1.00
- Recall = 1.00

This means the model has learned clear temporal patterns for emergency braking, not just thresholds.

⚠️ (3) The data distribution is still “easy”

Your concern was:

“The model has learned the data instead of understanding patterns.”

The correct refined statement is:

“The model understands patterns very well — but the patterns themselves are still too clean and consistent.”

This is a data realism issue, not a model issue. That’s an important distinction.

2️⃣ So… is this a problem?
❌ No, this is NOT a problem at this stage
✅ This is actually the expected outcome

Why?
- You deliberately started with clean synthetic data & controlled distributions
- This is baseline + first innovation validation

In real ML workflows:
- Validate architecture correctness ✅ (done)
- Validate training pipeline correctness ✅ (done)
- Validate controlled generalization ✅ (done)
- Then stress-test realism ❗ (next step)